# Set up Spark environment

In [1]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
        .master('local[*]')
        .appName('Intro to Spark')
        .config('spark.ui.port', '4050')
        .getOrCreate())
spark

# Esercitazione

Prima di iniziare con l'esercitazione carichiamo i file `people.csv` e `department.csv` nella cartella `content` e creiamo da essi il dataframe `people_df` e `department_df`.

In [2]:
people_df = spark.read.csv("people.csv", header=True, inferSchema=True)
assert people_df.count() == 1000
print("File people.csv caricato correttamente!")

File people.csv caricato correttamente!


Diamo un'occhiata a com'è composto il dataframe `people_df`, facendo uno `show`

In [3]:
people_df.show()

+---+----------+-------------+--------------------+----------+---------------+-----------------+-------------+--------------------+
| id|first_name|    last_name|               email|    gender|     ip_address|             city|      country|    ethereum_address|
+---+----------+-------------+--------------------+----------+---------------+-----------------+-------------+--------------------+
|  1|    Anatol|        Shave|      ashave0@a8.net|      Male|   32.94.154.64|  General Ramírez|    Argentina|0x1b9077156115717...|
|  2|     Marys|   Venediktov|mvenediktov1@nps.gov|    Female| 146.154.160.38|         Haapsalu|      Estonia|0xc7adcf05e5fe5cf...|
|  3|    Irvine|        Gaber|igaber2@slashdot.org|      Male|234.128.172.255|   Nyzhnya Krynka|      Ukraine|0x57a55596e577f2e...|
|  4|  Kristina|    Skitterel|kskitterel3@opens...|    Female|   105.61.14.18|        Mnelalete|    Indonesia|0x05178ef48e1bc63...|
|  5|  Modestia|      Minster|mminster4@sakura....|    Female| 135.235.86.10

Diamo un'occhiata allo schema del dataframe `people_df` facendo un `printSchema`. In questo modo potremo conoscere le colonne che compongono il dataframe, i relativi tipi e se possono o meno contenere valori null (nullable)

In [4]:
people_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- ethereum_address: string (nullable = true)



In [5]:
department_df = spark.read.csv("department.csv", header=True, inferSchema=True)
assert department_df.count() == 900
print("File department.csv caricato correttamente!")

File department.csv caricato correttamente!


In [6]:
department_df.show()

+---+--------------+
| id|    department|
+---+--------------+
|  1|     Marketing|
|  2|     Marketing|
|  3|Administration|
|  4|Administration|
|  5|            IT|
|  6|Administration|
|  7|       Finance|
|  8|     Marketing|
|  9|         Sales|
| 10|            IT|
| 11|Administration|
| 12|            IT|
| 13|            IT|
| 14|       Finance|
| 15|       Finance|
| 16|       Finance|
| 17|Administration|
| 18|       Finance|
| 19|         Sales|
| 20|         Sales|
+---+--------------+
only showing top 20 rows



In [7]:
department_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- department: string (nullable = true)



Eseguiamo questa cella per importare tutte le funzioni che ci serviranno per lavorare con i dataframe

In [8]:
from pyspark.sql.functions import *
from pyspark.sql import *

# Esercizio 1

Dal dataframe `people_df` selezioniamo le colonne `id` e `gender` e filtriamo per la colonna gender = Female, creando il dataframe `female_df`.

In [9]:
female_df = people_df.select('id', 'gender').filter(col("gender") == "Female")

In [10]:
assert female_df.schema.fieldNames() == ['id', 'gender'] 
assert female_df.count() == 455
print("Esercizio 1 superato!")

Esercizio 1 superato!


# Esercizio 2

Selezioniamo dal dataframe `people_df` le colonne `city` e `country` e aggiungiamo una nuova colonna `full_address` che sia il risultato della concatenazione delle colonne `city` e `country`, separate da uno spazio

In [11]:
full_address_df = people_df.select(col('city'), col('country'), concat_ws(' ', col('city'), col('country')).alias('full_address'))

In [12]:
assert full_address_df.schema.fieldNames() == ['city', 'country', 'full_address'] 
assert full_address_df.filter(col('city') == 'Haapsalu').select('full_address').collect()[0][0] == 'Haapsalu Estonia'
print("Esercizio 2 superato!")

Esercizio 2 superato!


# Esercizio 3

Dal dataframe `people_df` selezioniamo la colonna `ethereum_address` e filtriamo i record che iniziano con `0x1`, `0x2` e `0x3`.

Aggiungiamo poi una colonna `newCol`, popolata così:


1.   Se `ethereum_address` inizia per 0x1 metto a
2.   Se `ethereum_address` inizia per 0x2 metto b
3.   Se `ethereum_address` inizia per 0x3 metto c

Il daframe finale deve solo le colonne `ethereum_address` e `newCol`

In [13]:
ethereum_df = (people_df.select('ethereum_address', substring(col('ethereum_address'), 0, 3).alias('temp')) 
                    .filter(col('temp').isin('0x1', '0x2', '0x3')) 
                    .withColumn('newCol', when(col('temp') == '0x1', 'a').when(col('temp') == '0x2', 'b').otherwise('c')) 
                    .drop('temp'))

In [14]:
assert ethereum_df.schema.fieldNames() == ['ethereum_address', 'newCol'] 
assert ethereum_df.count() == 183
row = ethereum_df.filter(col('ethereum_address') == '0x1b907715611571700163387611121552d1ad1c9e').collect()[0]
assert row['newCol'] == 'a'
print("Esercizio 3 superato!")

Esercizio 3 superato!


# Esercizio 4

Dal dataframe `people_df` creiamo la colonna `newCol`come prima, facendo la substring su `ethereum_address` e prendiamo solo i record inclusi in `0x1`, `0x2` e `0x3`.

Poi calcoliamo quante righe ci sono per ogni casistica, in una nuova colonna `count`.

Per questo esercizio usiamo  `Spark SQL`.

Ricordiamo che prima di usare `spark.sql` e fare la query è necessario registrare il dataframe `people_df` col metodo `createOrReplaceTempView`.

Mi aspetto che il dataframe finale abbia solo due colonne `newCol` e `count`


In [39]:
people_df.createOrReplaceTempView('people_df')
sql_ethereum_df = (spark.sql("""
                      SELECT 
                          left(ethereum_address,3) AS newCol,
                          count(*) AS count
                      FROM people_df
                      WHERE left(ethereum_address,3) IN ('0x1','0x2','0x3')
                      GROUP BY newCol
                  """))

In [41]:
assert sql_ethereum_df.schema.fieldNames() == ['newCol', 'count'] 
assert sql_ethereum_df.count() == 3
row = sql_ethereum_df.filter(col('newCol') == '0x1').collect()[0]
assert row['count'] == 66
print("Esercizio 4 superato!")

Esercizio 4 superato!


# Esercizio 5

Dal dataframe `people_df` filtriamo i record con la colonna `gender` uguale a Male. 

Dopodichè calcoliamo una colonna `count` che contenga la count del numero di record per country e prendiamo la riga con la count maggiore.

In [63]:
male_df = (people_df.filter(col('gender') == 'Male')
                .groupBy('country')
                .agg(count("id").alias("count"))
                .orderBy("count", ascending = False))

In [64]:
assert male_df.schema.fieldNames() == ['country', 'count'] 
row_max =  male_df.collect()[0]
assert row_max['country'] == 'China'
assert row_max['count'] == 89
print("Esercizio 5 superato!")

Esercizio 5 superato!


# Esercizio 6

Il dataframe `people_df` contiene le informazioni sulle persone.

Il dataframe `department_df` contiene l'informazione del dipartimento in cui lavora ogni persona.

Ogni persona è individuata univocamente dall'`id` su entrambi i dataframe.

Adesso vogliamo arricchire il `people_df` con le informazioni del `department_df` ma mantenendo soltanto gli id delle persone presenti su entrambi i dataframe.

In [65]:
people_dept_df = people_df.join(department_df, 'id', 'inner')

In [66]:
assert people_dept_df.count() == 900
assert people_dept_df.schema.fieldNames() == ['id', 'first_name', 'last_name', 'email', 'gender', 'ip_address', 'city', 'country', 'ethereum_address', 'department']
print("Esercizio 6 superato!")

Esercizio 6 superato!


# Esercizio 7

Partendo dal dataframe appena calcolato `people_dept_df` dobbiamo calcolare per ogni department quante persone ci sono.

La colonna che conterrà il numero di persone la chiameremo `count`.

Prendiamo solo i department che hanno più di 200 persone.


In [67]:
count_people_dept_df = people_dept_df.groupBy('department').agg(count("id").alias("count")).filter(col('count') > 200)

In [68]:
assert count_people_dept_df.count() == 1
assert count_people_dept_df.schema.fieldNames() == ['department', 'count']
row = count_people_dept_df.collect()[0]
assert row['count'] == 203
print("Esercizio 7 superato!")

Esercizio 7 superato!


# Esercizio 8

Ripetere i passaggi dell'esercizio 7 ma usando `Spark SQL`

Ricordiamo che prima di usare `spark.sql` e fare la query è necessario registrare il dataframe `people_dept_df` col metodo `createOrReplaceTempView`. 

In [77]:
people_dept_df.createOrReplaceTempView("people_dept_df")
sql_count_people_dept_df = (spark.sql("""
                              SELECT
                                  department,
                                  count(*) AS count
                              FROM people_dept_df
                              GROUP BY department
                              HAVING count > 200
                            """))

In [78]:
assert sql_count_people_dept_df.count() == 1
assert sql_count_people_dept_df.schema.fieldNames() == ['department', 'count']
row = sql_count_people_dept_df.collect()[0]
assert row['count'] == 203
print("Esercizio 8 superato!")

Esercizio 8 superato!


# Esercizio 9

Da `people_df` filtriamo la colonna `ip_address` prendendo gli indirizzi ip che iniziano per `21`, dopodichè calcoliamo per quale `country` sono presenti più indirizzi ip. (Usiamo `pyspark.sql.Column.like`)

In [97]:
ip_address_per_country_df = (people_df.filter(col('ip_address').like('21%'))
        .groupBy('country')
        .agg(count("ip_address").alias("count"))
        .orderBy('count', ascending = False)
        .limit(1))                                    

In [98]:
assert ip_address_per_country_df.schema.fieldNames() == ['country', 'count']
row = ip_address_per_country_df.filter(col('country') == 'Indonesia').collect()[0]
assert row['count'] == 7
print("Esercizio 9 superato!")

Esercizio 9 superato!


# Esercizio 10

Da `people_df` filtriamo la colonna `email` considerando solo i valori che finiscono per `.com`. Poi facciamo una count aggregata per genere e estraiamo solo la prima riga contenente il numero maggiore di count.
(Usiamo `pyspark.sql.Column.rlike`)

In [100]:
email_per_gender_df = (people_df.filter(col('email').rlike('.com$'))
                        .groupBy('gender')
                        .agg(count("ip_address").alias("count"))
                        .orderBy('count', ascending = False)
                        .limit(1))   

In [101]:
assert email_per_gender_df.count() == 1
assert email_per_gender_df.schema.fieldNames() == ['gender', 'count']
row = email_per_gender_df.collect()[0]
assert row['gender'] == 'Female'
assert row['count'] == 275
print("Esercizio 10 superato!")

Esercizio 10 superato!
